In [ ]:
# !pip install deepeval
# !pip install python-dotenv
# !pip install rouge-score
# !pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=15b68232745607f03bff984a973ab32f607c8ba5855846a04fa6495a7c0a00ef
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import re
import numpy as np
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Importing metrics from deepeval
from deepeval.metrics.ragas import (
    RAGASContextualPrecisionMetric,
    RAGASFaithfulnessMetric,
    RAGASContextualRecallMetric,
    RAGASAnswerRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from deepeval.evaluate import evaluate

# Download necessary NLTK data files
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Import NLTK translation metrics
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

# Import ROUGE scorer
from rouge_score import rouge_scorer

## Evaluation Metrics Overview

This evaluation framework assesses our RAG system using both quantitative and qualitative metrics. The quantitative metrics provide objective, automated scores that reflect the similarity, consistency, and factuality of the generated answers. The qualitative metrics capture aspects such as context relevance, diversity, and clarity, which are essential for understanding the overall performance in real-world applications.

---

### Quantitative Metrics

- **BLEUMetric**  
  **What it measures:** Similarity between the generated answer and the expected output based on n-gram overlap.  
  **Explanation:** Uses NLTK's `sentence_bleu` to compute a BLEU score. A higher score indicates a better overlap with the reference answer.

- **ROUGEMetric**  
  **What it measures:** Recall-oriented overlap between the generated answer and the expected output.  
  **Explanation:** Computes ROUGE-1, ROUGE-2, and ROUGE-L F-measures and averages them. This reflects the degree to which key phrases and overall content are captured.

- **METEORMetric**  
  **What it measures:** Similarity between the generated answer and the expected output, addressing certain limitations of BLEU.  
  **Explanation:** Uses NLTK’s `meteor_score` to compute the METEOR score, providing an alternative perspective on answer quality.

- **FactualConsistencyMetric**  
  **What it measures:** The factual consistency between the generated answer and the retrieved context.  
  **Explanation:** Computes the cosine similarity between the TF-IDF vector representations of the generated answer and the combined retrieval context. A higher score indicates better alignment of the answer with the supporting context.

---

### Qualitative Metrics

- **RAGASContextualPrecisionMetric**  
  **What it measures:** Relevance of the retrieved context to the query.  
  **Threshold:** >0.7.

- **RAGASFaithfulnessMetric**  
  **What it measures:** Factual consistency between the generated answer and the retrieved context.  
  **Threshold:** >0.8.

- **RAGASContextualRecallMetric**  
  **What it measures:** Completeness of the retrieved context with respect to the query.  
  **Threshold:** >0.6.

- **RAGASAnswerRelevancyMetric**  
  **What it measures:** Relevance of the generated answer to the query.  
  **Threshold:** >0.7.

- **ContextualRelevancyMetric (Custom)**  
  **What it measures:** Combined relevance of the context and the generated answer to the query, computed via cosine similarity using TF-IDF vectors.  
  **Threshold:** >0.65.

- **ContextualDiversityMetric (Custom)**  
  **What it measures:** Diversity of the retrieved context.  
  **Threshold:** >0.5.

- **ContextualConsistencyMetric (Custom)**  
  **What it measures:** Consistency of the generated answer across multiple queries or context sources.  
  **Threshold:** >0.7.

- **TechnicalTermClarityMetric (Custom)**  
  **What it measures:** Clarity of technical term explanations in the generated answer.  
  **Threshold:** At least 1 explanation per technical term.

- **QuerySpecificityMetric (Custom)**  
  **What it measures:** Specificity of the generated answer relative to the query.  
  **Threshold:** >0.7.

- **CrossDocumentConsistencyMetric (Custom)**  
  **What it measures:** Consistency of the generated answer across multiple documents or context sources.  
  **Threshold:** >0.8.


In [ ]:
test_cases = [
    LLMTestCase(
        input="What is the role of attention mechanisms in Transformers?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="Attention mechanisms allow Transformers to focus on relevant parts of the input.",
        context=[
            """The matching engine processes these orders
through three sequential phases:
First, incoming limit orders that do not immediately cross the market are added to
the order book, maintaining strict price-time priority. Second, market orders are processed
through a two-stage matching algorithm: (a) market-to-market matching, where market
orders are netted against each other at the current market price, and (b) market-to-book
matching, where remaining market orders are executed against standing limit orders. Finally,
any crossing limit orders are matched against the order book."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    # Add more test cases
]

In [ ]:
test_cases = [
    LLMTestCase(
        input="Could you walk me through the three‑stage matching algorithm the authors implement in their continuous double‑auction market?",
        actual_output = """The matching engine in the continuous double-auction market uses a three-stage algorithm:
Stage 1: Order Book Population: Incoming limit orders (orders to buy or sell at a specific price or better) that don't immediately find a match are added to the order book. The order book is organized using price-time priority, meaning the best prices are matched first, and if multiple orders exist at the same price, the oldest order is prioritized.
Stage 2: Market Order Processing: Market orders (orders to buy or sell at the best available price) are processed in two steps:
(a) Market-to-Market Matching: Market buy orders are matched with market sell orders at the current market price. These orders are essentially netted against each other.
(b) Market-to-Book Matching: Any remaining market orders are then matched against existing limit orders in the order book.
Stage 3: Crossing Limit Order Matching: Any limit orders that cross the market (a buy order with a price higher than the best ask price or a sell order with a price lower than the best bid price) are matched against existing orders in the order book. This ensures that trades occur whenever possible.""",

        expected_output="""Orders are handled in three sequential phases: (1) non‑crossing limit orders join the book; (2a) market‑to‑market netting; (2b) market‑to‑book execution; (3) any still‑crossing limit orders are matched.""",
        context=[
            """The matching engine processes these orders
through three sequential phases:
First, incoming limit orders that do not immediately cross the market are added to
the order book, maintaining strict price-time priority. Second, market orders are processed
through a two-stage matching algorithm: (a) market-to-market matching, where market
orders are netted against each other at the current market price, and (b) market-to-book
matching, where remaining market orders are executed against standing limit orders. Finally,
any crossing limit orders are matched against the order book."""
            
        ],
        retrieval_context = [
    "value considerations.\nIn a double auction, buyers and sellers actively submit orders, with trades occurring when\ncompatible buy and sell orders match the price. The matching engine processes these orders\nthrough three sequential phases:\nFirst, incoming limit orders that do not immediately cross the market are added to\nthe order book, maintaining strict price-time priority. Second, market orders are processed\nthrough a two-stage matching algorithm: (a) market-to-market matching, where market\norders are netted against each other at the current market price, and (b) market-to-book\nmatching, where remaining market orders are executed against standing limit orders. Finally,\nany crossing limit orders are matched against the order book.\n10\n2.1.2 Implementation Details\nThe matching engine ( MatchingEngine class) implements three primary components that\nwork together to facilitate efficient market operation. The order processing component han-\ndles the core matching functionality, where market orders are executed immediately against\nthe best available prices in the order book during each trading round. When immediate\nexecution is impossible, limit orders are stored in the order book according to price-time\npriority. The system supports partial executions, maintaining careful tracking of remaining\nquantities to ensure complete order fulfillment across multiple trades when necessary.\nPosition management forms the second critical component, providing comprehensive\ntracking of agent positions and cash balances throughout the trading session. Before any\ntrade execution, the system performs rigorous pre-trade validation to ensure agents have suffi-\ncient resources to fulfill their orders. This check includes validating buyer cash commitments\nand shares availability for sellers, with the system maintaining accurate records of commit-\nted and available resources for each agent. The position management system updates in\nreal-time as trades are executed, ensuring market integrity and preventing over-commitment\nof resources.\nThe price formation mechanism constitutes the third component, implementing a sys-\ntematic price discovery and market monitoring approach. As trades are executed within each\nround, the system dynamically updates prices based on executed trades while continuously\ntracking market depth and bid-ask spreads. This data collection provides detailed insights\ninto market liquidity and efficiency. Each trade is logged with comprehensive information,\nincluding price, quantity, and participating agents, creating a complete audit trail of mar-\nket activity. This comprehensive price formation system ensures transparent price discovery\nwhile generating rich data for market quality analysis.\n11",
    "the second agents’ design, and the third the analysis module.\n8\n2.1 Market Design\nOur methodological framework integrates three components that create a controlled envi-\nronment for LLM-based trading agents. The framework implements a flexible continuous\ndouble-auction market mechanism that couples standard market microstructure principles\nwith market clearing and matching algorithms to accommodate asynchronous LLM decisions.\nThe market clearing process employs a dual-stage matching algorithm. In the first stage,\nlimit orders are posted. In the second stage, market orders are netted using a market-\nto-market matching engine that processes buy and sell orders, reconciling orders based on\navailable agent cash and share commitments. In the third stage, any remaining market\norders are matched against the existing order book, with unfilled quantities converted to\naggressive limit orders. This three-tiered approach optimizes immediate execution and price\ndiscovery while maintaining market liquidity.\nThe system’s OrderMatchingService orchestrates this process by coordinating trade ex-\necutions through the TradeExecutionService and managing order state transitions via the\nOrderStateManager. Each trade is logged in detail, with the overall market state—including\norder books, market depth, and price evolution—recalculated at the end of each trading\nround.\nThis modular design in the matching and clearing engine provides several advantages.\nFirst, it enables rigorous trade validation where each market order is validated against agent\ncash commitments and position constraints before matching, with the system dynamically\nadjusting order quantities based on available cash when an agent’s commitment is insuf-\nficiary, thus minimizing execution errors. Second, it offers flexible liquidity handling by\nsupporting market-to-market and market-to-book matching, ensuring orders have multiple\nexecution pathways, with unexecuted market orders converted to aggressive limit orders to\ncapture remaining liquidity. Third, it maintains detailed trade audit capabilities by recording\ncomprehensive traceability data including timestamps, trade volumes, executed prices, and\nagent identifiers, thereby enabling post-trade analysis and performance benchmarking that\n9",
    "2024). Another examines LLMs’ strategic capabilities through task-based simulations and\ncollaborative decision-making (Li, Zhang, and Sun 2023; Piatti et al. 2024).\nWithin the computer science literature, early works apply LLMs to financial markets but\nabstract away many fundamental stock market characteristics. Often, they employ single-\nprice clearing mechanisms without a persistent order book, ignore partial fills and bid-ask\nspreads, and omit dividends. Consequently, these simplified environments can primarily\naddress exogenous macro shocks (e.g., changes in interest rates or inflation; Gao et al. 2024),\nexogenous policy changes (Zhang et al. 2024), or rely on advanced methods such as repetitive\nnext-token predictions to generate better answers (Koa et al. 2024). This work extends\nthese efforts by incorporating these crucial market features, enabling the study of complex\nendogenous events such as flash crashes, liquidity shocks, and large-trader impacts.\nThis work also connects to emerging research on algorithmic and AI-powered trading\nsystems. Dou, Goldstein, and Ji (2024) demonstrate how reinforcement learning-based AI\nspeculators can autonomously learn to sustain collusive behavior without explicit coordi-\nnation, achieving supra-competitive profits through either price-trigger strategies or self-\nconfirming bias in learning. This finding is particularly informative for this framework as\nit highlights potential emergent behaviors that may arise when intelligent agents interact in\n7",
    "by detailed Trade Data, which records individual transaction details such as prices, quan-\ntities, buyer and seller identifiers, timestamps, round information, and specific execution\ndetails.\nAgent Data provides insights into participant behavior by monitoring cash and share\npositions, portfolio values, wealth evolution, and agent type and strategy identifiers. Order\nData captures the complete lifecycle of trading decisions, including order specifications\n(type, size, price), agent reasoning and justification, decision context and timing, and order\nlifecycle events. Finally, Market State data maintains a comprehensive view of overall mar-\nket conditions through order book snapshots, market depth information, aggregate statistics,\nand various market microstructure metrics.\n25\nFigure 5: Order Type Analysis under Systematic Variation. Displays the proportion of\nMarket vs. Limit orders used by different agent types across varying Price/Fundamental\nratios.\n26",
    "The theory of intermediation developed in Chapter 14 focuses on the risk-pooling and\nrisk-sharing products created by financial-services institutions. Products examined\nrange from standardized instruments such as a group of mutual funds with charac-\nteristics described in Theorem 15.5 to custom products tailored to meet the specific\nrequirements of each investor. Contingent-claims analysis (CCA) is the tool used\nto derive the production technologies and costs for intermediaries to create these\ncustom derivative securities. As discussed, such financial products and the associ-\nated activities of their producers are redundant in a frictionless environment with no\ninformation-gathering or transactions costs. Thus, to provide a nontrivial role for\nintermediaries in our equilibrium analysis here, we assume that some investors and\nbusiness firms face transactions costs for trading and issuing securities directly in\nthe capital markets. However, to preserve the CCA production theory for intermedi-\naries, we follow the model in Chapter 14 and posit that financial intermediaries, as\nthe lowest-cost transactors, can trade continuously in the capital market at no cost.\n16.2 Financial Intermediation with Dynamically-Complete Markets 417",
    "market efficiency and price discovery processes throughout the simulation.\nAgent performance metrics include absolute position tracking (shares, cash, and total\nwealth), wealth composition analysis, position change metrics, and return calculations. The\nsystem computes both absolute and percentage returns on various portfolio components\nand compares performance across agent types to identify relative advantages of different",
    "value, the system:\n1. Sets market price P = ρV where ρ is the target ratio\n2. Generates a consistent order book structure around this price\n3. Executes multiple decision trials with the same agent type\n4. Records decision outcomes (buy/sell/hold), order types, quantities, and reasoning",
    "allocations are briefly addressed in Chapter 15 and analyzed more fully in Chapter 16. If a set of existing\nsecurities and markets leads to an equilibrium that is an unconstrained Pareto optimum, then any additional\nsecurities or markets will be redundant.\n50The term “transactions costs” can, of course, mean more than just the bid-ask spread or commissions\npaid for buying and selling assets. For example, if an investor literally traded his portfolio continuously, he\nwould have no time to do anything else! Because investment management and financial intermediation involve\nprimarily information processing and security trading, there are significant economies of scale. One would thus\nexpect substantially lower costs for the individual investor to use intermediaries instead of transacting directly\nin the asset markets."
]
    ),
    LLMTestCase(
        input="Why do the authors choose discrete trading rounds instead of real‑time order processing for the LLM agents?",
        actual_output = """The authors use discrete trading rounds because LLMs (Large Language Models) have latency (delay) constraints, making real-time order processing infeasible. Essentially, LLMs take time to process information and make decisions, so a continuous real-time system wouldn't work efficiently. Discrete rounds provide structured intervals for the LLMs to receive information, decide on their actions, and submit their orders.""",

        expected_output="""LLMs have latency; discrete rounds let agents think while still approximating concurrency via randomised order of submission.""",
        context=[
            """When the market starts significantly above
the fundamental value (left panels), the price fails to converge downwards towards the $28.00
benchmark within the 15 rounds, remaining substantially elevated. Agent valuations mirror
this persistence, with many agents maintaining estimates well above the fundamental value.
However, when the market begins significantly below the fundamental value"""
            
        ],
        retrieval_context = [
    "value considerations.\nIn a double auction, buyers and sellers actively submit orders, with trades occurring when\ncompatible buy and sell orders match the price. The matching engine processes these orders\nthrough three sequential phases:\nFirst, incoming limit orders that do not immediately cross the market are added to\nthe order book, maintaining strict price-time priority. Second, market orders are processed\nthrough a two-stage matching algorithm: (a) market-to-market matching, where market\norders are netted against each other at the current market price, and (b) market-to-book\nmatching, where remaining market orders are executed against standing limit orders. Finally,\nany crossing limit orders are matched against the order book.\n10\n2.1.2 Implementation Details\nThe matching engine ( MatchingEngine class) implements three primary components that\nwork together to facilitate efficient market operation. The order processing component han-\ndles the core matching functionality, where market orders are executed immediately against\nthe best available prices in the order book during each trading round. When immediate\nexecution is impossible, limit orders are stored in the order book according to price-time\npriority. The system supports partial executions, maintaining careful tracking of remaining\nquantities to ensure complete order fulfillment across multiple trades when necessary.\nPosition management forms the second critical component, providing comprehensive\ntracking of agent positions and cash balances throughout the trading session. Before any\ntrade execution, the system performs rigorous pre-trade validation to ensure agents have suffi-\ncient resources to fulfill their orders. This check includes validating buyer cash commitments\nand shares availability for sellers, with the system maintaining accurate records of commit-\nted and available resources for each agent. The position management system updates in\nreal-time as trades are executed, ensuring market integrity and preventing over-commitment\nof resources.\nThe price formation mechanism constitutes the third component, implementing a sys-\ntematic price discovery and market monitoring approach. As trades are executed within each\nround, the system dynamically updates prices based on executed trades while continuously\ntracking market depth and bid-ask spreads. This data collection provides detailed insights\ninto market liquidity and efficiency. Each trade is logged with comprehensive information,\nincluding price, quantity, and participating agents, creating a complete audit trail of mar-\nket activity. This comprehensive price formation system ensures transparent price discovery\nwhile generating rich data for market quality analysis.\n11",
    "the second agents’ design, and the third the analysis module.\n8\n2.1 Market Design\nOur methodological framework integrates three components that create a controlled envi-\nronment for LLM-based trading agents. The framework implements a flexible continuous\ndouble-auction market mechanism that couples standard market microstructure principles\nwith market clearing and matching algorithms to accommodate asynchronous LLM decisions.\nThe market clearing process employs a dual-stage matching algorithm. In the first stage,\nlimit orders are posted. In the second stage, market orders are netted using a market-\nto-market matching engine that processes buy and sell orders, reconciling orders based on\navailable agent cash and share commitments. In the third stage, any remaining market\norders are matched against the existing order book, with unfilled quantities converted to\naggressive limit orders. This three-tiered approach optimizes immediate execution and price\ndiscovery while maintaining market liquidity.\nThe system’s OrderMatchingService orchestrates this process by coordinating trade ex-\necutions through the TradeExecutionService and managing order state transitions via the\nOrderStateManager. Each trade is logged in detail, with the overall market state—including\norder books, market depth, and price evolution—recalculated at the end of each trading\nround.\nThis modular design in the matching and clearing engine provides several advantages.\nFirst, it enables rigorous trade validation where each market order is validated against agent\ncash commitments and position constraints before matching, with the system dynamically\nadjusting order quantities based on available cash when an agent’s commitment is insuf-\nficiary, thus minimizing execution errors. Second, it offers flexible liquidity handling by\nsupporting market-to-market and market-to-book matching, ensuring orders have multiple\nexecution pathways, with unexecuted market orders converted to aggressive limit orders to\ncapture remaining liquidity. Third, it maintains detailed trade audit capabilities by recording\ncomprehensive traceability data including timestamps, trade volumes, executed prices, and\nagent identifiers, thereby enabling post-trade analysis and performance benchmarking that\n9",
    "2024). Another examines LLMs’ strategic capabilities through task-based simulations and\ncollaborative decision-making (Li, Zhang, and Sun 2023; Piatti et al. 2024).\nWithin the computer science literature, early works apply LLMs to financial markets but\nabstract away many fundamental stock market characteristics. Often, they employ single-\nprice clearing mechanisms without a persistent order book, ignore partial fills and bid-ask\nspreads, and omit dividends. Consequently, these simplified environments can primarily\naddress exogenous macro shocks (e.g., changes in interest rates or inflation; Gao et al. 2024),\nexogenous policy changes (Zhang et al. 2024), or rely on advanced methods such as repetitive\nnext-token predictions to generate better answers (Koa et al. 2024). This work extends\nthese efforts by incorporating these crucial market features, enabling the study of complex\nendogenous events such as flash crashes, liquidity shocks, and large-trader impacts.\nThis work also connects to emerging research on algorithmic and AI-powered trading\nsystems. Dou, Goldstein, and Ji (2024) demonstrate how reinforcement learning-based AI\nspeculators can autonomously learn to sustain collusive behavior without explicit coordi-\nnation, achieving supra-competitive profits through either price-trigger strategies or self-\nconfirming bias in learning. This finding is particularly informative for this framework as\nit highlights potential emergent behaviors that may arise when intelligent agents interact in\n7",
    "by detailed Trade Data, which records individual transaction details such as prices, quan-\ntities, buyer and seller identifiers, timestamps, round information, and specific execution\ndetails.\nAgent Data provides insights into participant behavior by monitoring cash and share\npositions, portfolio values, wealth evolution, and agent type and strategy identifiers. Order\nData captures the complete lifecycle of trading decisions, including order specifications\n(type, size, price), agent reasoning and justification, decision context and timing, and order\nlifecycle events. Finally, Market State data maintains a comprehensive view of overall mar-\nket conditions through order book snapshots, market depth information, aggregate statistics,\nand various market microstructure metrics.\n25\nFigure 5: Order Type Analysis under Systematic Variation. Displays the proportion of\nMarket vs. Limit orders used by different agent types across varying Price/Fundamental\nratios.\n26",
    "The theory of intermediation developed in Chapter 14 focuses on the risk-pooling and\nrisk-sharing products created by financial-services institutions. Products examined\nrange from standardized instruments such as a group of mutual funds with charac-\nteristics described in Theorem 15.5 to custom products tailored to meet the specific\nrequirements of each investor. Contingent-claims analysis (CCA) is the tool used\nto derive the production technologies and costs for intermediaries to create these\ncustom derivative securities. As discussed, such financial products and the associ-\nated activities of their producers are redundant in a frictionless environment with no\ninformation-gathering or transactions costs. Thus, to provide a nontrivial role for\nintermediaries in our equilibrium analysis here, we assume that some investors and\nbusiness firms face transactions costs for trading and issuing securities directly in\nthe capital markets. However, to preserve the CCA production theory for intermedi-\naries, we follow the model in Chapter 14 and posit that financial intermediaries, as\nthe lowest-cost transactors, can trade continuously in the capital market at no cost.\n16.2 Financial Intermediation with Dynamically-Complete Markets 417",
    "market efficiency and price discovery processes throughout the simulation.\nAgent performance metrics include absolute position tracking (shares, cash, and total\nwealth), wealth composition analysis, position change metrics, and return calculations. The\nsystem computes both absolute and percentage returns on various portfolio components\nand compares performance across agent types to identify relative advantages of different",
    "value, the system:\n1. Sets market price P = ρV where ρ is the target ratio\n2. Generates a consistent order book structure around this price\n3. Executes multiple decision trials with the same agent type\n4. Records decision outcomes (buy/sell/hold), order types, quantities, and reasoning",
    "allocations are briefly addressed in Chapter 15 and analyzed more fully in Chapter 16. If a set of existing\nsecurities and markets leads to an equilibrium that is an unconstrained Pareto optimum, then any additional\nsecurities or markets will be redundant.\n50The term “transactions costs” can, of course, mean more than just the bid-ask spread or commissions\npaid for buying and selling assets. For example, if an investor literally traded his portfolio continuously, he\nwould have no time to do anything else! Because investment management and financial intermediation involve\nprimarily information processing and security trading, there are significant economies of scale. One would thus\nexpect substantially lower costs for the individual investor to use intermediaries instead of transacting directly\nin the asset markets."
]
    ),
    LLMTestCase(
        input="What is the main experimental finding when the simulated market starts 25 % above the fundamental value?",
        actual_output="""In the infinite horizon scenario where the simulated market began 25% above the fundamental value (overvalued at $56.00 compared to the true value of $28.00), the price failed to converge downwards toward the fundamental value within the 15 trading rounds. The LLM (Large Language Model - a type of artificial intelligence model that can generate human-like text) agents' valuations also remained significantly elevated above the fundamental value, mirroring the persistent overvaluation in the market. This suggests that, under these specific conditions, the AI agents were less effective at correcting overvaluation compared to undervaluation within the given timeframe.""",
        expected_output="""With an initial price 25 % above fundamentals, prices failed to converge downward within 15 rounds and agents kept over‑valuing the asset.""",
        context=[
            """When the market starts significantly above the fundamental value … the price fails to converge downwards … remaining substantially elevated. Agent valuations mirror this persistence, with many agents maintaining estimates well above the fundamental value."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    
    LLMTestCase(
        input="What is the main experimental finding when the simulated market starts 25 % above the fundamental value?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""With an initial price 25 % above fundamentals, prices failed to converge downward within 15 rounds and agents kept over‑valuing the asset.""",
        context=[
            """When the market starts significantly above the fundamental value … the price fails to converge downwards … remaining substantially elevated. Agent valuations mirror this persistence, with many agents maintaining estimates well above the fundamental value."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="In simple terms, what does Figure 7 (agent valuations over time) tell us about over‑ vs under‑valuation corrections?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""It shows asymmetry: over‑valuation persists, but under‑valuation is corrected—the right panel converges to $28 while the left stays high.""",
        context=[
            """When the market starts significantly above the fundamental value … the price fails to converge downwards … remaining substantially elevated. Agent valuations mirror this persistence, with many agents maintaining estimates well above the fundamental value."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="The paper gives Vt=E[D]rV_t=\tfrac{E[D]}{r} as Equation (1). What does each variable represent, and how is this perpetuity formula used in the simulations?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""Vt=E[D]/rV_t = E[D]/r treats the stock as a perpetuity: E[D]E[D] is expected dividend per round and rr the risk‑free rate; with $1.40 and 5 % the value is $28.""",
        context=[
            """The framework implements a consistent approach… For infinite horizon markets, the fundamental value follows the standard dividend discount model: Vt=E[D]rV_t = \frac{E[D]}{r}. … For the default parameters where the expected dividend is $1.40 and the interest rate is 5 %, this yields a fundamental value of $28.00."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="What is a ‘continuous double auction’, and how does it differ from the single‑price clearing used in earlier LLM‑market papers the authors critique?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""It allows buyers and sellers to post bids/asks continuously with price‑time priority, unlike prior LLM papers that used single‑price batch clearing without an order book.""",
        context=[
            """Our framework implements a continuous double‑auction market…” + “Within the computer‑science literature, early works … employ single‑price clearing mechanisms without a persistent order book…"""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="How do the authors define the price‑to‑fundamental ratio and why is it central to their systematic variation experiments?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""ρ = P ⁄ V. By sweeping ρ from 0.1–3.5 and holding other factors fixed, the authors trace decision boundaries of each agent type.""",
        context=[
            """Our example implementation varies the price‑to‑fundamental ratio (ρ = P/V) across a range from 0.1 to 3.5 while maintaining all other market parameters constant…"""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="HWhat are the roles of system prompts vs user prompts in shaping an LLM trading agent’s behaviour?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""System prompt fixes the agent’s identity and objectives across rounds; user prompt feeds current market state & constraints for the immediate decision.""",
        context=[
            """The system prompt establishes the agent’s fundamental trading characteristics…” / “The user prompt provides the immediate market context necessary for tactical decision‑making."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="The authors note that LLMs ‘follow instructions even if doing so results in financial losses.’ How might this limit their realism as profit‑seeking traders?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""LLMs optimise for prompt fidelity; they keep executing the instructed strategy even when it loses money, unlike humans who adapt for profit.""",
        context=[
            """They maintain their strategic direction even if doing so results in financial losses. This… creates a unique trading profile distinct from rule‑based algorithms and human traders."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="What systemic risks do the authors foresee if many firms deploy similarly prompted LLM traders in real markets?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""If many firms deploy similarly prompted LLMs built on the same foundation model, their synchronised reactions could amplify volatility and create new instabilities.""",
        context=[
            """A key concern is the potential for widespread correlated behavior… similar underlying LLM architectures responding uniformly… could destabilize trading patterns without explicit coordination."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="In SRCNN, what practical role does the non‑linear mapping layer with 1 × 1 filters play in the overall SR pipeline?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""It transforms every patch‑level feature vector (extracted by the first conv layer) into a new, lower‑dimensional representation that will later be linearly combined into HR pixels. Because the spatial kernel is 1 × 1, the mapping is applied channel‑wise without mixing spatial positions, mimicking the sparse‑coding step but in a single feed‑forward pass.""",
        context=[
            """The first layer extracts an n₁‑dimensional feature for each patch. In the second operation, we **map each of these n₁‑dimensional vectors into an n₂‑dimensional one. This is equivalent to applying n₂ filters which have a trivial spatial support 1 × 1 … This operation non‑linearly maps each high‑dimensional vector onto another high‑dimensional vector."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="Why do the authors of SRCNN first upscale the LR image with bicubic interpolation instead of learning that step end‑to‑end?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""Because popular GPU conv‑net toolkits (cuda‑convnet at the time) did not yet support fractional‑stride convolutions. Upscaling once with bicubic ensures the network can stay fully convolutional with integer strides while still learning the rest of the SR mapping end‑to‑end.""",
        context=[
            """…we first upscale it to the desired size using bicubic interpolation, which is the only pre‑processing we perform. … There is a fractional stride. To take advantage of the popular well‑optimized implementations such as cuda‑convnet, we exclude this “layer” from learning."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="What exactly is meant by “perceptual loss” in SRGAN, and how is it computed from VGG feature maps?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""It is a weighted sum of a content loss (Euclidean distance between VGG‑feature maps of SR vs. HR images) and an adversarial loss (−log D(G(I))). The VGG layers (e.g., φ5,4) provide semantics; the adversarial term pushes outputs onto the natural‑image manifold.""",
        context=[
            """We formulate the perceptual loss as the weighted sum of a content loss and an adversarial loss component … Content loss l_{VGG/i.j} = 1/(W_{i,j}H_{i,j}) Σ(φ_{i,j}(I_{HR}) − φ_{i,j}(G(I_{LR})))² … l_{Gen} = −log D(G(I_{LR}))"""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="SRCNN reports 8 032 trainable parameters; can you explain how this relatively small count is achieved?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""Because it uses just three convolutional layers with relatively few filters (64, 32, c) and small kernels (9×9, 1×1, 5×5). Param count = (c·9²·64)+(64·1²·32)+(32·5²·c)+biases ≈ 8 k""",
        context=[
            """A typical and basic setting is f₁ = 9, f₂ = 1, f₃ = 5, n₁ = 64, n₂ = 32 … the number of parameters of 9‑1‑5 … is 8 032."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    LLMTestCase(
        input="SRGAN introduces two networks called SRResNet and SRGAN. What is the structural difference between them?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="""Both share the same 16‑block residual generator; SRResNet is trained only with a pixel/feature content loss, whereas SRGAN adds a discriminator and the adversarial term during training, yielding sharper textures.""",
        context=[
            """We employ a deep residual network (SRResNet) … We then train SRGAN, which uses the same generator but adds a discriminator network and optimizes the adversarial loss in addition to the content loss."""
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    # Add more test cases
]

In [ ]:
class BLEUMetric:
    """
    BLEU Metric calculates the BLEU score between the generated answer (actual_output)
    and the expected output as a measure of similarity.
    
    A higher BLEU score indicates better overlap between the generated answer and the reference.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        reference = [test_case.expected_output.split()]
        hypothesis = test_case.actual_output.split()
        smoothing = SmoothingFunction().method1
        self.score = sentence_bleu(reference, hypothesis, smoothing_function=smoothing)
        return self.score

class ROUGEMetric:
    """
    ROUGE Metric calculates ROUGE scores (e.g., ROUGE-1, ROUGE-2, ROUGE-L) between the
    generated answer (actual_output) and the expected output.
    
    These scores reflect the recall-oriented overlap between the generated and reference texts.
    """
    def __init__(self):
        self.score = 0
        self.details = {}

    def measure(self, test_case: LLMTestCase) -> float:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(test_case.expected_output, test_case.actual_output)
        # For simplicity, we can average the three ROUGE scores (using F-measure)
        self.details = scores
        self.score = np.mean([score.fmeasure for score in scores.values()])
        return self.score

class METEORMetric:
    """
    METEOR Metric calculates the METEOR score between the generated answer (actual_output)
    and the expected output. METEOR is designed to address some weaknesses of BLEU.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        self.score = meteor_score([test_case.expected_output], test_case.actual_output)
        return self.score

class FactualConsistencyMetric:
    """
    FactualConsistencyMetric measures the factual consistency between the generated answer 
    and the retrieved context by computing the cosine similarity (using TF-IDF vectors)
    between the answer and the combined retrieval context.
    
    A higher score indicates a better alignment of the generated answer with the context.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        answer = test_case.actual_output
        context = " ".join(test_case.retrieval_context)
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([answer, context])
        similarity = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        self.score = similarity
        return self.score

In [4]:
class ContextualRelevancyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        query = test_case.input
        answer = test_case.actual_output
        context = " ".join(test_case.retrieval_context)
        
        # Compute similarity between query, answer, and context
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([query, answer, context])
        query_answer_sim = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        query_context_sim = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]
        
        self.score = (query_answer_sim + query_context_sim) / 2
        return self.score
    


class ContextualDiversityMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        
        # Compute pairwise similarity between contexts
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(contexts)
        similarity_matrix = cosine_similarity(vectors)
        
        # Average similarity (lower = more diverse)
        avg_similarity = np.mean(similarity_matrix[np.triu_indices(len(contexts), 1)])
        self.score = 1 - avg_similarity
        return self.score
    

class ContextualConsistencyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        answer = test_case.actual_output
        
        # Check if answer appears in all contexts
        consistent = all([answer in ctx for ctx in contexts])
        self.score = 1.0 if consistent else 0.0
        return self.score
    

class TechnicalTermClarityMetric:
    def __init__(self):
        self.score = 0
        self.explanations = []

    def measure(self, test_case: LLMTestCase) -> float:
        answer = test_case.actual_output
        # Regex to detect terms with explanations in parentheses
        pattern = r"([A-Z][a-zA-Z\s]+) \((.+?)\)"
        matches = re.findall(pattern, answer)
        
        self.explanations = [term[0] for term in matches]
        self.score = len(matches) / len(self._detect_technical_terms(answer))
        return self.score

    def _detect_technical_terms(self, text: str) -> List[str]:
        # Replace with your domain-specific term list
        technical_terms = ["attention mechanisms", "Transformers", "self-attention"]
        return [term for term in technical_terms if term in text]
    

class QuerySpecificityMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        query = test_case.input
        answer = test_case.actual_output
        
        # Simple heuristic: Longer answers for specific queries
        query_words = len(query.split())
        answer_words = len(answer.split())
        self.score = min(answer_words / (query_words * 3), 1.0)
        return self.score
    

class CrossDocumentConsistencyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        answer = test_case.actual_output
        
        # Check if answer appears in all contexts
        consistent = all([answer in ctx for ctx in contexts])
        self.score = 1.0 if consistent else 0.0
        return self.score
    


In [ ]:
def evaluate_rag_system(test_cases):
    metrics = [
        RAGASContextualPrecisionMetric(threshold=0.7),
        RAGASFaithfulnessMetric(threshold=0.8),
        RAGASContextualRecallMetric(threshold=0.6),
        RAGASAnswerRelevancyMetric(threshold=0.7),
        ContextualRelevancyMetric(),
        ContextualDiversityMetric(),
        ContextualConsistencyMetric(),
        TechnicalTermClarityMetric(),
        QuerySpecificityMetric(),
        CrossDocumentConsistencyMetric(),
        # New quantitative metrics:
        BLEUMetric(),
        ROUGEMetric(),
        METEORMetric(),
        FactualConsistencyMetric()
    ]
    
    return evaluate(test_cases, metrics)

In [ ]:
results = evaluate_rag_system(test_cases)
for metric in results:
    metric_name = metric.__class__.__name__
    # For ROUGEMetric, include additional details if available.
    if metric_name == "ROUGEMetric":
        print(f"{metric_name}: {metric.score:.3f} (Details: {metric.details}) (Passed: {metric.score >= 0.0})")
    else:
        print(f"{metric_name}: {metric.score:.3f} (Passed: {metric.score >= getattr(metric, 'threshold', 0)})")